In [ ]:
import cv2
import numpy as np
from scipy import integrate

def windmill(frame_width, frame_height, fps, duration, num_blades, direction, mask='both'):
    num_frames = int(duration * fps)
    frames = []

    def wind_ve(x):
        return np.sin(2 * np.pi * x / 5) * direction

    for frame_count in range(num_frames):
        frame = np.zeros((frame_width, frame_height, 3), dtype=np.uint8)

        frame_t = frame_count / fps
        angle = integrate.quad(wind_ve, 0, frame_t)[0] * 180 / np.pi

        center = (frame_width // 2, frame_height // 2)
        for j in range(0, 360, int(360 / num_blades * 2)):
            start_angle = -90 + angle
            cv2.ellipse(frame, center, center, 0, start_angle + j, start_angle + j + 20, (255, 255, 255), -1)

        if mask == 'left':
            frame[:, frame_width // 2:] = 0
        elif mask == 'right':
            frame[:, :frame_width // 2] = 0

        frames.append(frame)

    return frames

def generate_black_white_frames(frame_width, frame_height, black_duration, white_duration, fps):
    num_black_frames = int(black_duration * fps)
    num_white_frames = int(white_duration * fps)

    # 创建黑色和白色帧
    black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    white_frame = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255

    frames = []

    # 生成白色帧
    for _ in range(num_white_frames):
        frames.append(white_frame.copy())

    # 生成黑色帧
    for _ in range(num_black_frames):
        frames.append(black_frame.copy())

    return frames


def grating(direction,v,t,hz,width,bar_width,real_width):  # width设为600比较好,V:cm/s,real-width:cm, direction=forward,backward,left,right
    
    grating_cycle=[]
    step=v/real_width*width/hz  #每帧应跨像素数     
    
    #最小循环单元
    n=0
    for i in np.arange(0,int(bar_width*2),step):
        n=n+1
        img = np.ones((width, width, 3), np.uint8) * 255
        for j in range(0,int(width/bar_width/2+2)):
            if direction=="forward":
                cv2.line(img, (0, int(j*2*bar_width-i+bar_width/2)), (width, int(j*2*bar_width-i+bar_width/2)), (0,0,0), bar_width)
            if direction=="reverse":
                cv2.line(img, (0, int((j-2)*2*bar_width+i+bar_width/2)), (width, int((j-2)*2*bar_width+i+bar_width/2)), (0,0,0), bar_width)
            if direction=="left":
                cv2.line(img, (int(j*2*bar_width-i+bar_width/2), 0), (int(j*2*bar_width-i+bar_width/2), width), (0,0,0), bar_width)   
            if direction=="right":
                cv2.line(img, (int((j-2)*2*bar_width+i+bar_width/2), 0), (int((j-2)*2*bar_width+i+bar_width/2), width), (0,0,0), bar_width)
        grating_cycle.append(img)
    
            
    #以最小循环单元为基础的循环数和余数
    cycle=int(t*hz/n)
    cycle_tail=t*hz % n

    img_arra=[]
    for q in range(0,cycle):  #
        img_arra=img_arra+grating_cycle
    img_sti=img_arra + grating_cycle[0:cycle_tail]
    
    return img_sti

def blank(inter,d,hz):  #空白帧 （时间，尺寸，帧率）
      # sec
    img_inter=[]
    for k in range(0,int(inter*hz)):  
        img=np.ones((d, d, 3), np.uint8) * 0
        img_inter.append(img)
    return img_inter


forward_03=grating("forward",v=0.3,t=5,hz=60,width=600,bar_width=60,real_width=5)
forward_1=grating("forward",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
forward_3=grating("forward",v=3,t=5,hz=60,width=600,bar_width=60,real_width=5)
reverse=grating("reverse",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
left=grating("left",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
right=grating("right",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
f_w=windmill(600, 600, 60, 5, 18, 1, mask='both')
r_w=windmill(600, 600, 60, 5, 18, -1, mask='both')
f_wR=windmill(600, 600, 60, 5, 18, 1, mask='right')
r_wR=windmill(600, 600, 60, 5, 18, -1, mask='right')
f_wl=windmill(600, 600, 60, 5, 18, 1, mask='left')
r_wl=windmill(600, 600, 60, 5, 18, -1, mask='left')
flash=generate_black_white_frames(600, 600, 1, 1, 60)
m_flash=flash+flash+flash+flash




Cycle = []

# 视频参数
output_width = 600
output_height = 600
fps = 60
duration_per_sequence = 5  # 每个序列的持续时间（秒）
gap_duration = 5  # 两个序列之间的间隔时间（秒）

# 将每个帧序列写入视频
sequences = [forward_03, forward_1, forward_3, reverse, left, right, f_w, r_w, f_wR, r_wl, m_flash]
for i, sequence in enumerate(sequences):
    num_frames = len(sequence)
    Cycle += sequence

    # 最后一个序列不生成间隔帧
    if i < len(sequences) - 2:
        # 间隔时间，显示上一个序列的最后一帧和下一个序列的第一帧
        gap_frames = int(gap_duration * fps)
        last_frame = sequence[-1]
        next_sequence = sequences[i + 1]
        next_frame = next_sequence[0]
        for _ in range(gap_frames // 2):
            Cycle.append(last_frame)
        for _ in range(gap_frames // 2):
            Cycle.append(next_frame)
    elif i == len(sequences) - 2:
        gap_frames = int(2.5 * fps)
        last_frame = sequence[-1]
        for _ in range(gap_frames):
            Cycle.append(last_frame)
        Cycle += blank(2.5,output_width,fps)
        
final = blank(10,output_width,fps) + Cycle + blank(10,output_width,fps) + Cycle + blank(10,output_width,fps)

# 创建 VideoWriter 对象
out = cv2.VideoWriter('output_final.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (output_width, output_height))
# 保存视频
for frame in final:
    out.write(frame)
# 释放 VideoWriter 对象
out.release()